# Review Analysis Pipeline
### This notebook performs comprehensive analysis on review data including:
### - Data cleaning and deduplication
### - Review text similarity analysis
### - User rating behavior analysis
### - Location category similarity analysis
### - User posting frequency analysis
### - Statistical significance testing for ratings

# DATA LOADING AND INITIAL CLEANING

- Loads user metadata from 'user_metadata.csv' into a DataFrame.
- Removes duplicates based on 'user_name', 'review_text', and location coordinates.
- Drops rows with missing 'review_text' and resets index.
- Prints review counts before/after deduplication and NaN removal.
- Displays cleaned DataFrame's first few rows.

In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Load the data
all_reviews = pd.read_csv('final_data/user_metadata.csv')
print("Initial data shape:")
all_reviews.head()

# Remove duplicates based on user, review text, and location
print(f'Before deduplication: {len(all_reviews)} reviews')
all_reviews = all_reviews.drop_duplicates(subset=['user_name', 'review_text', 'location_latitude', 'location_longitude'])
print(f'After deduplication: {len(all_reviews)} reviews')

# Remove rows with missing review text
all_reviews = all_reviews.dropna(subset=['review_text']).reset_index(drop=True)
print(f'After removing NaN review_text: {len(all_reviews)} reviews')

all_reviews.head()


Initial data shape:
Before deduplication: 117034 reviews
After deduplication: 116226 reviews
After removing NaN review_text: 116226 reviews


,user_id,user_name,review_rating,review_text,location_name,location_latitude,location_longitude,location_category,location_average_rating,location_reviews_count,review_policy,review_fake,review_years_since_posted,user_reviews_count
0,1.100000e+20,Michelle Banks,5.0,It's a beautiful place to read books and have ...,Barnes & Noble,36.157754,-115.289418,"['Book store', 'Cafe', 'Childrens book store',...",4.6,1719.0,relevant,no,7,148.0
1,1.060000e+20,Steven DeRyck [Staff],4.0,"As previous reviews have stated, two small pie...",Carnegie Deli,36.120556,-115.173611,"['Deli', 'Takeout Restaurant', 'Sandwich shop']",4.1,706.0,relevant,no,6,143.0
2,1.100000e+20,Stevey Markovich,5.0,Absolutely love this office! Afton is truly am...,Center for Cosmetic and Family Dentistry,36.001929,-115.107484,['Dentist'],4.9,318.0,relevant,no,4,148.0
3,1.020000e+20,William Campbell,3.0,The food is as good as it usually is,Asian Garden,36.168901,-115.060601,"['Restaurant', 'Asian restaurant', 'Chinese re...",3.8,128.0,relevant,no,6,145.0
4,1.120000e+20,Beverly Thorman,5.0,We came in without an appointment on a Saturda...,Great Clips,36.191055,-115.258969,"['Hair salon', 'Beauty salon']",4.3,168.0,relevant,no,7,138.0


# CONFIGURATION PARAMETERS

In [14]:
# Minimum number of reviews required for similarity analysis
MIN_REVIEWS_FOR_CHECK = 3

# Parameters for statistical significance testing
Z_SCORE = 2.58   # stricter threshold (~99% confidence)
MIN_DIFF = 1.0   # require at least 1-star difference
MAX_STD = 2      # since ratings are bounded [1,5]

# TEXT PREPROCESSING AND REVIEW SIMILARITY ANALYSIS

- Cleans review text (lowercase, remove punctuation/numbers, normalize spaces).
- Computes minimum cosine similarity for each user's reviews using TF-IDF.
- Adds preprocessed text, merges similarity scores into DataFrame, removes temp column.
- Prints status, user count with scores, and DataFrame preview.

In [15]:
def preprocess_text(text):
    """
    Cleans the input text by lowercasing, removing punctuation, numbers, and extra spaces.
    
    Args:
        text: Raw text string
        
    Returns:
        Cleaned text string
    """
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Keep only letters and spaces
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

def get_min_user_review_similarity(review_texts):
    """
    Calculates pairwise cosine similarity for a list of reviews and returns the
    minimum similarity score among all pairs.
    
    Args:
        review_texts: List of review text strings
        
    Returns:
        Minimum pairwise similarity score or np.nan if insufficient data
    """
    if len(review_texts) < MIN_REVIEWS_FOR_CHECK:
        return np.nan

    vectorizer = TfidfVectorizer(stop_words='english')
    
    try:
        tfidf_matrix = vectorizer.fit_transform(review_texts)
        if tfidf_matrix.shape[0] < MIN_REVIEWS_FOR_CHECK or tfidf_matrix.nnz == 0:
            return np.nan
    except ValueError:
        return np.nan

    cosine_sim_matrix = cosine_similarity(tfidf_matrix)

    # Extract all pairwise similarities (upper triangle)
    pairwise_similarities = []
    for i in range(len(review_texts)):
        for j in range(i + 1, len(review_texts)):
            pairwise_similarities.append(cosine_sim_matrix[i, j])
    
    if not pairwise_similarities:
        return np.nan

    return np.min(pairwise_similarities)

# Apply text preprocessing
print("Preprocessing review texts...")
all_reviews['cleaned_review_text'] = all_reviews['review_text'].apply(preprocess_text)

# Calculate review similarity for each user
print("Calculating review similarity for each user...")
user_min_similarities = {}
for user_id, group in all_reviews.groupby('user_id'):
    reviews_for_user = group['cleaned_review_text'].tolist()
    user_min_similarities[user_id] = get_min_user_review_similarity(reviews_for_user)

# Merge similarity scores back to main DataFrame
user_similarity_df = pd.DataFrame(user_min_similarities.items(), 
                                 columns=['user_id', 'review_similarity'])
all_reviews = pd.merge(all_reviews, user_similarity_df, on='user_id', how='left')

# Clean up temporary column
all_reviews = all_reviews.drop(columns=['cleaned_review_text'])

print("Review similarity analysis completed.")
print(f"Users with similarity scores: {all_reviews['review_similarity'].notna().sum()}")
all_reviews.head()

Preprocessing review texts...
Calculating review similarity for each user...
Review similarity analysis completed.
Users with similarity scores: 106344


,user_id,user_name,review_rating,review_text,location_name,location_latitude,location_longitude,location_category,location_average_rating,location_reviews_count,review_policy,review_fake,review_years_since_posted,user_reviews_count,review_similarity
0,1.100000e+20,Michelle Banks,5.0,It's a beautiful place to read books and have ...,Barnes & Noble,36.157754,-115.289418,"['Book store', 'Cafe', 'Childrens book store',...",4.6,1719.0,relevant,no,7,148.0,0.0
1,1.060000e+20,Steven DeRyck [Staff],4.0,"As previous reviews have stated, two small pie...",Carnegie Deli,36.120556,-115.173611,"['Deli', 'Takeout Restaurant', 'Sandwich shop']",4.1,706.0,relevant,no,6,143.0,0.0
2,1.100000e+20,Stevey Markovich,5.0,Absolutely love this office! Afton is truly am...,Center for Cosmetic and Family Dentistry,36.001929,-115.107484,['Dentist'],4.9,318.0,relevant,no,4,148.0,0.0
3,1.020000e+20,William Campbell,3.0,The food is as good as it usually is,Asian Garden,36.168901,-115.060601,"['Restaurant', 'Asian restaurant', 'Chinese re...",3.8,128.0,relevant,no,6,145.0,0.0
4,1.120000e+20,Beverly Thorman,5.0,We came in without an appointment on a Saturda...,Great Clips,36.191055,-115.258969,"['Hair salon', 'Beauty salon']",4.3,168.0,relevant,no,7,138.0,0.0


# USER RATING BEHAVIOR ANALYSIS

- Groups reviews by 'user_id' to calculate mean and standard deviation of ratings.
- Merges rating stats (average and std) into the main DataFrame.
- Prints completion status and shows updated DataFrame preview.

In [16]:
print("Analyzing user rating behavior...")

# Calculate average and standard deviation of ratings per user
user_rating_stats = (
    all_reviews.groupby('user_id')['review_rating']
    .agg(['mean', 'std'])
    .reset_index()
    .rename(columns={'mean': 'avg_review_rating_by_user', 'std': 'std_review_rating_by_user'})
)

# Merge rating statistics back to main DataFrame
all_reviews = pd.merge(all_reviews, user_rating_stats, on='user_id', how='left')

print("User rating behavior analysis completed.")
all_reviews.head()

Analyzing user rating behavior...
User rating behavior analysis completed.


,user_id,user_name,review_rating,review_text,location_name,location_latitude,location_longitude,location_category,location_average_rating,location_reviews_count,review_policy,review_fake,review_years_since_posted,user_reviews_count,review_similarity,avg_review_rating_by_user,std_review_rating_by_user
0,1.100000e+20,Michelle Banks,5.0,It's a beautiful place to read books and have ...,Barnes & Noble,36.157754,-115.289418,"['Book store', 'Cafe', 'Childrens book store',...",4.6,1719.0,relevant,no,7,148.0,0.0,4.320354,1.188759
1,1.060000e+20,Steven DeRyck [Staff],4.0,"As previous reviews have stated, two small pie...",Carnegie Deli,36.120556,-115.173611,"['Deli', 'Takeout Restaurant', 'Sandwich shop']",4.1,706.0,relevant,no,6,143.0,0.0,4.155598,1.368606
2,1.100000e+20,Stevey Markovich,5.0,Absolutely love this office! Afton is truly am...,Center for Cosmetic and Family Dentistry,36.001929,-115.107484,['Dentist'],4.9,318.0,relevant,no,4,148.0,0.0,4.320354,1.188759
3,1.020000e+20,William Campbell,3.0,The food is as good as it usually is,Asian Garden,36.168901,-115.060601,"['Restaurant', 'Asian restaurant', 'Chinese re...",3.8,128.0,relevant,no,6,145.0,0.0,4.167969,1.325318
4,1.120000e+20,Beverly Thorman,5.0,We came in without an appointment on a Saturda...,Great Clips,36.191055,-115.258969,"['Hair salon', 'Beauty salon']",4.3,168.0,relevant,no,7,138.0,0.0,4.213084,1.262917


# LOCATION CATEGORY SIMILARITY ANALYSIS

- Calculates minimum cosine similarity for each user's location categories using TF-IDF.
- Groups reviews by 'user_id', computes similarity, merges scores into DataFrame.
- Prints status and DataFrame preview.

In [17]:
def get_min_user_category_similarity(categories):
    """
    Calculates pairwise cosine similarity for a list of categories and returns
    the minimum similarity score among all pairs.
    
    Args:
        categories: List of category strings
        
    Returns:
        Minimum pairwise similarity score or np.nan if insufficient data
    """
    if len(categories) < MIN_REVIEWS_FOR_CHECK:
        return np.nan

    vectorizer = TfidfVectorizer(stop_words='english')
    
    try:
        tfidf_matrix = vectorizer.fit_transform(categories)
        if tfidf_matrix.shape[0] < MIN_REVIEWS_FOR_CHECK or tfidf_matrix.nnz == 0:
            return np.nan
    except ValueError:
        return np.nan

    cosine_sim_matrix = cosine_similarity(tfidf_matrix)

    # Extract all pairwise similarities
    pairwise_similarities = []
    for i in range(len(categories)):
        for j in range(i + 1, len(categories)):
            pairwise_similarities.append(cosine_sim_matrix[i, j])

    if not pairwise_similarities:
        return np.nan

    return np.min(pairwise_similarities)

print("Analyzing location category diversity for each user...")

# Calculate category similarity for each user
user_category_similarities = {}
for user_id, group in all_reviews.groupby('user_id'):
    categories_for_user = group['location_category'].astype(str).tolist()
    user_category_similarities[user_id] = get_min_user_category_similarity(categories_for_user)

# Merge category similarity scores back to main DataFrame
user_category_sim_df = pd.DataFrame(user_category_similarities.items(), 
                                   columns=['user_id', 'category_similarity'])
all_reviews = pd.merge(all_reviews, user_category_sim_df, on='user_id', how='left')

print("Location category similarity analysis completed.")
all_reviews.head()

Analyzing location category diversity for each user...
Location category similarity analysis completed.


,user_id,user_name,review_rating,review_text,location_name,location_latitude,location_longitude,location_category,location_average_rating,location_reviews_count,review_policy,review_fake,review_years_since_posted,user_reviews_count,review_similarity,avg_review_rating_by_user,std_review_rating_by_user,category_similarity
0,1.100000e+20,Michelle Banks,5.0,It's a beautiful place to read books and have ...,Barnes & Noble,36.157754,-115.289418,"['Book store', 'Cafe', 'Childrens book store',...",4.6,1719.0,relevant,no,7,148.0,0.0,4.320354,1.188759,0.0
1,1.060000e+20,Steven DeRyck [Staff],4.0,"As previous reviews have stated, two small pie...",Carnegie Deli,36.120556,-115.173611,"['Deli', 'Takeout Restaurant', 'Sandwich shop']",4.1,706.0,relevant,no,6,143.0,0.0,4.155598,1.368606,0.0
2,1.100000e+20,Stevey Markovich,5.0,Absolutely love this office! Afton is truly am...,Center for Cosmetic and Family Dentistry,36.001929,-115.107484,['Dentist'],4.9,318.0,relevant,no,4,148.0,0.0,4.320354,1.188759,0.0
3,1.020000e+20,William Campbell,3.0,The food is as good as it usually is,Asian Garden,36.168901,-115.060601,"['Restaurant', 'Asian restaurant', 'Chinese re...",3.8,128.0,relevant,no,6,145.0,0.0,4.167969,1.325318,0.0
4,1.120000e+20,Beverly Thorman,5.0,We came in without an appointment on a Saturda...,Great Clips,36.191055,-115.258969,"['Hair salon', 'Beauty salon']",4.3,168.0,relevant,no,7,138.0,0.0,4.213084,1.262917,0.0


# USER POSTING FREQUENCY ANALYSIS

- Converts 'review_years_since_posted' to numeric, handling errors.
- Calculates posting frequency (posts per year) by grouping reviews by 'user_id', counting total reviews, and dividing by active years (minimum 1 to avoid division by zero).
- Merges posting frequency into the main DataFrame.
- Prints completion status and shows updated DataFrame preview.

In [18]:
print("Analyzing user posting frequency...")

# Ensure review_years_since_posted is numeric
all_reviews['review_years_since_posted'] = pd.to_numeric(all_reviews['review_years_since_posted'], errors='coerce')

# Calculate posting frequency (posts per year) for each user
user_frequency = (
    all_reviews.groupby('user_id')
    .agg(
        total_reviews=('review_years_since_posted', 'count'),
        active_years=('review_years_since_posted', lambda x: (x.max() - x.min() + 1))
    )
    .reset_index()
)

# Avoid divide by zero if user only posted in 1 year
user_frequency['posts_per_year'] = (user_frequency['total_reviews'] / 
                                   user_frequency['active_years'].replace(0, 1))

# Merge posting frequency back to main DataFrame
all_reviews = pd.merge(all_reviews, user_frequency[['user_id', 'posts_per_year']], 
                      on='user_id', how='left')

print("User posting frequency analysis completed.")
all_reviews.head()

Analyzing user posting frequency...
User posting frequency analysis completed.


,user_id,user_name,review_rating,review_text,location_name,location_latitude,location_longitude,location_category,location_average_rating,location_reviews_count,review_policy,review_fake,review_years_since_posted,user_reviews_count,review_similarity,avg_review_rating_by_user,std_review_rating_by_user,category_similarity,posts_per_year
0,1.100000e+20,Michelle Banks,5.0,It's a beautiful place to read books and have ...,Barnes & Noble,36.157754,-115.289418,"['Book store', 'Cafe', 'Childrens book store',...",4.6,1719.0,relevant,no,7,148.0,0.0,4.320354,1.188759,0.0,43.461538
1,1.060000e+20,Steven DeRyck [Staff],4.0,"As previous reviews have stated, two small pie...",Carnegie Deli,36.120556,-115.173611,"['Deli', 'Takeout Restaurant', 'Sandwich shop']",4.1,706.0,relevant,no,6,143.0,0.0,4.155598,1.368606,0.0,37.642857
2,1.100000e+20,Stevey Markovich,5.0,Absolutely love this office! Afton is truly am...,Center for Cosmetic and Family Dentistry,36.001929,-115.107484,['Dentist'],4.9,318.0,relevant,no,4,148.0,0.0,4.320354,1.188759,0.0,43.461538
3,1.020000e+20,William Campbell,3.0,The food is as good as it usually is,Asian Garden,36.168901,-115.060601,"['Restaurant', 'Asian restaurant', 'Chinese re...",3.8,128.0,relevant,no,6,145.0,0.0,4.167969,1.325318,0.0,46.545455
4,1.120000e+20,Beverly Thorman,5.0,We came in without an appointment on a Saturda...,Great Clips,36.191055,-115.258969,"['Hair salon', 'Beauty salon']",4.3,168.0,relevant,no,7,138.0,0.0,4.213084,1.262917,0.0,44.583333


# STATISTICAL SIGNIFICANCE TESTING FOR RATINGS

- Computes SEM using `MAX_STD` and `location_reviews_count`.
- Flags ratings as significantly different if they exceed statistical (Z_SCORE * SEM) and practical (`MIN_DIFF`) thresholds from location average.
- Adds boolean column `rating_significantly_different` to DataFrame.
- Prints completion status and distribution of significant differences.

In [19]:
print("Performing statistical significance testing for individual ratings...")

# Calculate standard error of the mean (SEM) bound
all_reviews['sem_max'] = MAX_STD / np.sqrt(all_reviews['location_reviews_count'].clip(lower=1))

# Determine if individual ratings are significantly different from location average
all_reviews['rating_significantly_different'] = (
    # Check if difference exceeds statistical threshold
    ((all_reviews['review_rating'] - all_reviews['location_average_rating']).abs() > 
     Z_SCORE * all_reviews['sem_max']) &
    # Check if difference meets minimum practical threshold
    ((all_reviews['review_rating'] - all_reviews['location_average_rating']).abs() >= MIN_DIFF)
)

print("Statistical significance testing completed.")
print("\nDistribution of significantly different ratings:")
print(all_reviews['rating_significantly_different'].value_counts())

Performing statistical significance testing for individual ratings...
Statistical significance testing completed.

Distribution of significantly different ratings:
rating_significantly_different
False    83577
True     32649
Name: count, dtype: int64


# CREATE FINAL USER-LEVEL SUMMARY DATASET

- Groups reviews by `user_id`, aggregating features: first value for `review_similarity`, `category_similarity`, `avg_review_rating_by_user`, `std_review_rating_by_user`, `posts_per_year`; sums `rating_significantly_different`.
- Renames `rating_significantly_different` to `count_significantly_different_ratings`.
- Reorders columns as specified.
- Prints dataset status, shape, columns, feature completeness, and DataFrame preview.

In [ ]:
print("Creating final user-level feature dataset...")

# Group by user_id and aggregate all the features we've created
user_features = all_reviews.groupby('user_id').agg({
    'review_similarity': 'first',  # Same value for all rows of a user
    'category_similarity': 'first',  # Same value for all rows of a user
    'avg_review_rating_by_user': 'first',  # Same value for all rows of a user
    'std_review_rating_by_user': 'first',  # Same value for all rows of a user
    'posts_per_year': 'first',  # Same value for all rows of a user
    'rating_significantly_different': 'sum'  # Count of significantly different ratings per user
}).reset_index()

# Rename the aggregated column to be clearer
user_features = user_features.rename(columns={
    'std_review_rating': 'std_review_rating_by_user',
    'rating_significantly_different': 'count_significantly_different_ratings'
})

# Reorder columns to match your requested output
final_columns = [
    'user_id',
    'review_similarity', 
    'category_similarity',
    'avg_review_rating_by_user',
    'std_review_rating_by_user',
    'posts_per_year',
    'count_significantly_different_ratings'  # Changed from boolean to count
]

user_features = user_features[final_columns]

print(f"\nFinal user features dataset created!")
print(f"Shape: {user_features.shape}")
print(f"Columns: {list(user_features.columns)}")

print(f"\nFeature completeness:")
for col in user_features.columns:
    if col != 'user_id':
        non_null_count = user_features[col].notna().sum()
        print(f"- {col}: {non_null_count:,} users ({non_null_count/len(user_features)*100:.1f}%)")

user_features.head()

Creating final user-level feature dataset...

Final user features dataset created!
Shape: (16394, 7)
Columns: ['user_id', 'review_similarity', 'category_similarity', 'avg_review_rating_by_user', 'std_review_rating_by_user', 'posts_per_year', 'count_significantly_different_ratings']

Feature completeness:
- review_similarity: 7,450 users (45.4%)
- category_similarity: 7,451 users (45.4%)
- avg_review_rating_by_user: 16,394 users (100.0%)
- std_review_rating_by_user: 8,289 users (50.6%)
- posts_per_year: 16,394 users (100.0%)
- count_significantly_different_ratings: 16,394 users (100.0%)


,user_id,review_similarity,category_similarity,avg_review_rating_by_user,std_review_rating_by_user,posts_per_year,count_significantly_different_ratings
0,1.000000e+20,0.0,0.000000,4.098101,1.396093,31.600000,101
1,1.000000e+20,0.0,0.098012,4.400000,0.966092,5.000000,1
2,1.000009e+20,0.0,0.000000,5.000000,0.000000,8.000000,0
3,1.000013e+20,0.0,0.572069,4.500000,0.577350,4.000000,1
4,1.000017e+20,0.0,0.203046,4.571429,0.786796,2.333333,5


# Merge Dataset with Reviews Dataset

In [ ]:
df_reviews = pd.read_csv('final_data/feature_engineered_dataset_reviews.csv')
merged_df = pd.merge(df_reviews, user_features, on='user_id', how='left')

# Save Dataset

In [23]:
merged_df.to_csv("final_data/final_dataset.csv", index=False, encoding='utf-8-sig')